In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 200

1. Data Preprocessing (Check Null , Data Type , See The correct Form ofthe data if there any white space for example or not)
2. Check And Handel The Outliers Using (EX : IQR OR Z SCORE ) 
3. Feature Engineering ( Feature Selection , Feature Extraction)
4. Transformation For The Categorical Data 
5. Train Test Split
6. Modeling And Accuracy Calculation

In [37]:
df = pd.read_csv('first inten project.csv')
df.head()

,Booking_ID,number of adults,number of children,number of weekend nights,number of week nights,type of meal,car parking space,room type,lead time,market segment type,repeated,P-C,P-not-C,average price,special requests,date of reservation,booking status
0,INN00001,1,1,2,5,Meal Plan 1,0,Room_Type 1,224,Offline,0,0,0,88.00,0,10/2/2015,Not_Canceled
1,INN00002,1,0,1,3,Not Selected,0,Room_Type 1,5,Online,0,0,0,106.68,1,11/6/2018,Not_Canceled
2,INN00003,2,1,1,3,Meal Plan 1,0,Room_Type 1,1,Online,0,0,0,50.00,0,2/28/2018,Canceled
3,INN00004,1,0,0,2,Meal Plan 1,0,Room_Type 1,211,Online,0,0,0,100.00,1,5/20/2017,Canceled
4,INN00005,1,0,1,2,Not Selected,0,Room_Type 1,48,Online,0,0,0,77.00,0,4/11/2018,Canceled


In [38]:
df = df.rename(columns={
    'P-C': 'reserved then cancelled',
    'P-not-C': 'reserved then not cancelled',
    'lead time': 'days from booking to arrival',
    'repeated': 'visited before',
    'average price ': 'average price',
    'market segment type': 'market type'})

df.head()

,Booking_ID,number of adults,number of children,number of weekend nights,number of week nights,type of meal,car parking space,room type,days from booking to arrival,market type,visited before,reserved then cancelled,reserved then not cancelled,average price,special requests,date of reservation,booking status
0,INN00001,1,1,2,5,Meal Plan 1,0,Room_Type 1,224,Offline,0,0,0,88.00,0,10/2/2015,Not_Canceled
1,INN00002,1,0,1,3,Not Selected,0,Room_Type 1,5,Online,0,0,0,106.68,1,11/6/2018,Not_Canceled
2,INN00003,2,1,1,3,Meal Plan 1,0,Room_Type 1,1,Online,0,0,0,50.00,0,2/28/2018,Canceled
3,INN00004,1,0,0,2,Meal Plan 1,0,Room_Type 1,211,Online,0,0,0,100.00,1,5/20/2017,Canceled
4,INN00005,1,0,1,2,Not Selected,0,Room_Type 1,48,Online,0,0,0,77.00,0,4/11/2018,Canceled


In [39]:
df.duplicated().sum()

0

numerical data: number of adults, number of children, number of weekend nights	, number of week nights , avg price , days from booking to arrival , reserved then not cancelled , reserved then cancelled , special requests

In [40]:
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower) | (data[column] > upper)]
    return outliers

print("Available columns:", df.columns.tolist())

columns_to_check = [
    'number of adults',
    'number of children',
    'number of weekend nights',
    'number of week nights',
    'days from booking to arrival',
    'reserved then cancelled',
    'reserved then not cancelled',
    'special requests',
    'average price'
]

for i in columns_to_check:
        outliers = detect_outliers_iqr(df, i)
        print(f'Outliers in "{i}": {len(outliers)} rows')


Available columns: ['Booking_ID', 'number of adults', 'number of children', 'number of weekend nights', 'number of week nights', 'type of meal', 'car parking space', 'room type', 'days from booking to arrival', 'market type', 'visited before', 'reserved then cancelled', 'reserved then not cancelled', 'average price', 'special requests', 'date of reservation', 'booking status']
Outliers in "number of adults": 10175 rows
Outliers in "number of children": 2702 rows
Outliers in "number of weekend nights": 21 rows
Outliers in "number of week nights": 324 rows
Outliers in "days from booking to arrival": 1332 rows
Outliers in "reserved then cancelled": 338 rows
Outliers in "reserved then not cancelled": 812 rows
Outliers in "special requests": 762 rows
Outliers in "average price": 1696 rows


In [41]:
df.describe()

,number of adults,number of children,number of weekend nights,number of week nights,car parking space,days from booking to arrival,visited before,reserved then cancelled,reserved then not cancelled,average price,special requests
count,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000
mean,1.844839,0.105360,0.810693,2.204602,0.030977,85.239851,0.025630,0.023343,0.153369,103.421636,0.619733
std,0.518813,0.402704,0.870590,1.410946,0.173258,85.938796,0.158032,0.368281,1.753931,35.086469,0.786262
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,0.000000,17.000000,0.000000,0.000000,0.000000,80.300000,0.000000
50%,2.000000,0.000000,1.000000,2.000000,0.000000,57.000000,0.000000,0.000000,0.000000,99.450000,0.000000
75%,2.000000,0.000000,2.000000,3.000000,0.000000,126.000000,0.000000,0.000000,0.000000,120.000000,1.000000
max,4.000000,10.000000,7.000000,17.000000,1.000000,443.000000,1.000000,13.000000,58.000000,540.000000,5.000000


In [42]:
#clipping outliers using IQR method
def cap_outliers_iqr(data, columns):
    for col in columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        num_below = (data[col] < lower).sum()
        num_above = (data[col] > upper).sum()

        data[col] = data[col].clip(lower, upper)

        print(f'[{col}] - Capped {num_below} below and {num_above} above to IQR limits.')

    return data


df = cap_outliers_iqr(df, columns_to_check)


[number of adults] - Capped 7841 below and 2334 above to IQR limits.
[number of children] - Capped 0 below and 2702 above to IQR limits.
[number of weekend nights] - Capped 0 below and 21 above to IQR limits.
[number of week nights] - Capped 0 below and 324 above to IQR limits.
[days from booking to arrival] - Capped 0 below and 1332 above to IQR limits.
[reserved then cancelled] - Capped 0 below and 338 above to IQR limits.
[reserved then not cancelled] - Capped 0 below and 812 above to IQR limits.
[special requests] - Capped 0 below and 762 above to IQR limits.
[average price] - Capped 627 below and 1069 above to IQR limits.


In [43]:
df.dtypes

Booking_ID                       object
number of adults                  int64
number of children                int64
number of weekend nights          int64
number of week nights             int64
type of meal                     object
car parking space                 int64
room type                        object
days from booking to arrival    float64
market type                      object
visited before                    int64
reserved then cancelled           int64
reserved then not cancelled       int64
average price                   float64
special requests                float64
date of reservation              object
booking status                   object
dtype: object

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36285 entries, 0 to 36284
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Booking_ID                    36285 non-null  object 
 1   number of adults              36285 non-null  int64  
 2   number of children            36285 non-null  int64  
 3   number of weekend nights      36285 non-null  int64  
 4   number of week nights         36285 non-null  int64  
 5   type of meal                  36285 non-null  object 
 6   car parking space             36285 non-null  int64  
 7   room type                     36285 non-null  object 
 8   days from booking to arrival  36285 non-null  float64
 9   market type                   36285 non-null  object 
 10  visited before                36285 non-null  int64  
 11  reserved then cancelled       36285 non-null  int64  
 12  reserved then not cancelled   36285 non-null  int64  
 13  a

In [45]:
df.columns

Index(['Booking_ID', 'number of adults', 'number of children',
       'number of weekend nights', 'number of week nights', 'type of meal',
       'car parking space', 'room type', 'days from booking to arrival',
       'market type', 'visited before', 'reserved then cancelled',
       'reserved then not cancelled', 'average price', 'special requests',
       'date of reservation', 'booking status'],
      dtype='object')

transfer categorical data into numerical

In [46]:
df['type of meal'].value_counts()

type of meal
Meal Plan 1     27842
Not Selected     5132
Meal Plan 2      3306
Meal Plan 3         5
Name: count, dtype: int64

In [47]:
df.replace({'type of meal': {'Not Selected': 0,
                               'Meal Plan 1': 1,
                                'Meal Plan 2': 2,
                                'Meal Plan 3': 3}}, inplace=True)

df['type of meal'].value_counts()

C:\Users\dell\AppData\Local\Temp\ipykernel_19724\1181922773.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'type of meal': {'Not Selected': 0,


type of meal
1    27842
0     5132
2     3306
3        5
Name: count, dtype: int64

In [48]:
df['room type'].value_counts()

room type
Room_Type 1    28138
Room_Type 4     6059
Room_Type 6      966
Room_Type 2      692
Room_Type 5      265
Room_Type 7      158
Room_Type 3        7
Name: count, dtype: int64

In [49]:
df.replace({'room type': {'Room_Type 1': 1,
                                'Room_Type 2': 2,
                                'Room_Type 3': 3,
                                'Room_Type 4': 4,
                                'Room_Type 5': 5,
                                'Room_Type 6': 6,
                                'Room_Type 7': 7,}}, inplace=True)


df['room type'].value_counts()

C:\Users\dell\AppData\Local\Temp\ipykernel_19724\377456246.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'room type': {'Room_Type 1': 1,


room type
1    28138
4     6059
6      966
2      692
5      265
7      158
3        7
Name: count, dtype: int64

In [50]:
df['market type'].value_counts()

market type
Online           23221
Offline          10531
Corporate         2017
Complementary      391
Aviation           125
Name: count, dtype: int64

In [51]:
df.replace({'market type': {'Online': 1,
                                'Offline': 2,
                                'Corporate': 3,
                                'Complementary': 4,
                                'Aviation': 5,
                           }}, inplace=True)


df['market type'].value_counts()

C:\Users\dell\AppData\Local\Temp\ipykernel_19724\750805983.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'market type': {'Online': 1,


market type
1    23221
2    10531
3     2017
4      391
5      125
Name: count, dtype: int64

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36285 entries, 0 to 36284
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Booking_ID                    36285 non-null  object 
 1   number of adults              36285 non-null  int64  
 2   number of children            36285 non-null  int64  
 3   number of weekend nights      36285 non-null  int64  
 4   number of week nights         36285 non-null  int64  
 5   type of meal                  36285 non-null  int64  
 6   car parking space             36285 non-null  int64  
 7   room type                     36285 non-null  int64  
 8   days from booking to arrival  36285 non-null  float64
 9   market type                   36285 non-null  int64  
 10  visited before                36285 non-null  int64  
 11  reserved then cancelled       36285 non-null  int64  
 12  reserved then not cancelled   36285 non-null  int64  
 13  a

In [53]:
df['date of reservation'] = pd.to_datetime(df['date of reservation'], errors='coerce')
df.head()

,Booking_ID,number of adults,number of children,number of weekend nights,number of week nights,type of meal,car parking space,room type,days from booking to arrival,market type,visited before,reserved then cancelled,reserved then not cancelled,average price,special requests,date of reservation,booking status
0,INN00001,2,0,2,5,1,0,1,224.0,2,0,0,0,88.00,0.0,2015-10-02,Not_Canceled
1,INN00002,2,0,1,3,0,0,1,5.0,1,0,0,0,106.68,1.0,2018-11-06,Not_Canceled
2,INN00003,2,0,1,3,1,0,1,1.0,1,0,0,0,50.00,0.0,2018-02-28,Canceled
3,INN00004,2,0,0,2,1,0,1,211.0,1,0,0,0,100.00,1.0,2017-05-20,Canceled
4,INN00005,2,0,1,2,0,0,1,48.0,1,0,0,0,77.00,0.0,2018-04-11,Canceled


In [54]:
invalid_dates = df[df['date of reservation'].isna()]
print(f"Number of corrupted or invalid dates: {len(invalid_dates)}")

df = df.dropna(subset=['date of reservation'])

Number of corrupted or invalid dates: 37


In [55]:
df['reservation_day'] = df['date of reservation'].dt.day
df['reservation_month'] = df['date of reservation'].dt.month
df['reservation_weekday'] = df['date of reservation'].dt.weekday 

df.drop(columns=['date of reservation'], inplace=True)
df.head()


,Booking_ID,number of adults,number of children,number of weekend nights,number of week nights,type of meal,car parking space,room type,days from booking to arrival,market type,visited before,reserved then cancelled,reserved then not cancelled,average price,special requests,booking status,reservation_day,reservation_month,reservation_weekday
0,INN00001,2,0,2,5,1,0,1,224.0,2,0,0,0,88.00,0.0,Not_Canceled,2,10,4
1,INN00002,2,0,1,3,0,0,1,5.0,1,0,0,0,106.68,1.0,Not_Canceled,6,11,1
2,INN00003,2,0,1,3,1,0,1,1.0,1,0,0,0,50.00,0.0,Canceled,28,2,2
3,INN00004,2,0,0,2,1,0,1,211.0,1,0,0,0,100.00,1.0,Canceled,20,5,5
4,INN00005,2,0,1,2,0,0,1,48.0,1,0,0,0,77.00,0.0,Canceled,11,4,2


In [56]:

df.replace({'booking status': {'Canceled': 0,
                                'Not_Canceled': 1}}, inplace=True)

df['booking status'].value_counts()

C:\Users\dell\AppData\Local\Temp\ipykernel_19724\806945440.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'booking status': {'Canceled': 0,


booking status
1    24366
0    11882
Name: count, dtype: int64

In [57]:
df.drop(columns=['Booking_ID'], inplace=True)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36248 entries, 0 to 36284
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   number of adults              36248 non-null  int64  
 1   number of children            36248 non-null  int64  
 2   number of weekend nights      36248 non-null  int64  
 3   number of week nights         36248 non-null  int64  
 4   type of meal                  36248 non-null  int64  
 5   car parking space             36248 non-null  int64  
 6   room type                     36248 non-null  int64  
 7   days from booking to arrival  36248 non-null  float64
 8   market type                   36248 non-null  int64  
 9   visited before                36248 non-null  int64  
 10  reserved then cancelled       36248 non-null  int64  
 11  reserved then not cancelled   36248 non-null  int64  
 12  average price                 36248 non-null  float64
 13  specia

In [59]:
df['weekends to total'] = df['number of weekend nights'] * 1.0 / (df['number of weekend nights'] + df['number of week nights'])
df['weekdays to total'] = df['number of week nights'] * 1.0 / (df['number of weekend nights'] + df['number of week nights'])
df['weekends to total'] = df['weekends to total'].fillna(0) 
df['weekdays to total'] = df['weekdays to total'].fillna(0)


df.drop(columns=['number of weekend nights', 'number of week nights'], inplace=True)

df['weekends to total'].value_counts()

weekends to total
0.000000    16876
0.500000     4226
0.333333     4213
0.666667     2466
0.250000     2310
1.000000     2305
0.400000     1700
0.285714      974
0.200000      743
0.166667      310
0.454545       55
0.375000       53
0.444444       17
Name: count, dtype: int64

check multicolleniarity

In [60]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

X = add_constant(df)  
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
import pandas as pd

vif_df = pd.DataFrame({
    'Feature': X.columns,
    'VIF': vif
})

print(vif_df)



c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


                         Feature         VIF
0               number of adults  491.064989
1             number of children         NaN
2                   type of meal    1.252183
3              car parking space    1.031772
4                      room type    1.300352
5   days from booking to arrival    1.447256
6                    market type    1.623239
7                 visited before    1.190724
8        reserved then cancelled         NaN
9    reserved then not cancelled         NaN
10                 average price    1.659762
11              special requests    1.300109
12                booking status    1.465647
13               reservation_day    1.004577
14             reservation_month    1.049832
15           reservation_weekday    1.282070
16             weekends to total   41.625368
17             weekdays to total   41.588796


In [61]:
df.nunique()

number of adults                   1
number of children                 1
type of meal                       4
car parking space                  2
room type                          7
days from booking to arrival     291
market type                        5
visited before                     2
reserved then cancelled            1
reserved then not cancelled        1
average price                   3503
special requests                   4
booking status                     2
reservation_day                   31
reservation_month                 12
reservation_weekday                7
weekends to total                 13
weekdays to total                 13
dtype: int64

In [62]:
df.drop(columns=[
    'number of adults',
    'number of children',
    'reserved then cancelled',
    'reserved then not cancelled',
    'weekdays to total' 
], inplace=True)

In [63]:
X = df.drop(columns=['booking status'])
y = df['booking status']

In [64]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

In [65]:
from sklearn.model_selection import train_test_split

X_train , X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

In [66]:
#knn model

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report


knn_model = KNeighborsClassifier(n_neighbors=5 , weights='distance', algorithm='auto')
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.87
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.78      0.80      2366
           1       0.90      0.91      0.91      4884

    accuracy                           0.87      7250
   macro avg       0.86      0.85      0.85      7250
weighted avg       0.87      0.87      0.87      7250



In [67]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=5000)
logistic_model.fit(X_train, y_train)
y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Accuracy: {accuracy:.2f}")
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred))

Logistic Regression Accuracy: 0.80
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.62      0.67      2366
           1       0.83      0.89      0.86      4884

    accuracy                           0.80      7250
   macro avg       0.78      0.75      0.76      7250
weighted avg       0.79      0.80      0.79      7250



In [68]:
from sklearn.tree import DecisionTreeClassifier

decision_tree_model = DecisionTreeClassifier(random_state=0)
decision_tree_model.fit(X_train, y_train)
y_pred = decision_tree_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Decision Tree Accuracy: {accuracy:.2f}")
print("Decision Tree Classification Report:")
print(classification_report(y_test, y_pred))

Decision Tree Accuracy: 0.87
Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.81      0.80      2366
           1       0.91      0.90      0.90      4884

    accuracy                           0.87      7250
   macro avg       0.85      0.86      0.85      7250
weighted avg       0.87      0.87      0.87      7250



In [69]:
#random forest
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(random_state=0)
random_forest_model.fit(X_train, y_train)
y_pred = random_forest_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy: {accuracy:.2f}")
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred))

Random Forest Accuracy: 0.90
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.81      0.84      2366
           1       0.91      0.95      0.93      4884

    accuracy                           0.90      7250
   macro avg       0.90      0.88      0.89      7250
weighted avg       0.90      0.90      0.90      7250

